In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import datetime
%matplotlib inline 

In [ ]:
# load the data
path_ed = '/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/'
path_m = '/media/kpierce/seagate/reverting-onecell/analysis/m-stats/'
path_cdf = '/media/kpierce/seagate/reverting-onecell/analysis/rt-cdf/'
efile = 'flow_a-l_0.6-time_1000.0hr-entrains.npy'
dfile = 'flow_a-l_0.6-time_1000.0hr-deposits.npy'
mfile = 'flow_a-l_0.6-time_1000.0hr-mstats.dat'
cfile = 'flow_a-l_0.6-time-1000.0hr-rtcdf.npy'

In [ ]:
# define parameters
tmax = 500*3600 # maximum return time to allow for (seconds) 
dt = 0.5 # bin size in seconds
bins = np.arange(0, tmax, dt) # bins over which to histogram

In [ ]:
def compute_crt(te, me, td, md, mv, bins = bins):
    """ 
    compute conditional return time given
    - entrainment timeseries te
    - bed elevations me at entrainment
    - deposition timeseries td
    - bed elevations md at deposition
    - a value mv at which to condition
    - a set of bins in time over which to histogram 
    """
    returns = te[me==mv]
    departs = td[md==mv+1]
    
    if (len(returns)>0) and (len(departs)>0): 
        while (returns[0]<departs[0]):
            returns=returns[1:]
        while (departs[-1]>returns[-1]):
            departs=departs[:-1]
        k = len(departs)-len(returns)
        if k>0:
            departs=departs[k:]
        elif k<0:
            returns=returns[:k]
        crt = returns-departs
        H,_ = np.histogram(crt,bins=bins,density=True)
        F = 1-H.cumsum()*(bins[1]-bins[0])
    else:
        F = np.zeros(len(bins)-1,dtype=float)
    return F

In [ ]:
def loader(mfile):
    path_m = '/media/kpierce/seagate/reverting-onecell/analysis/m-stats/'
    path_ed = '/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/'
    path_c = '/media/kpierce/seagate/reverting-onecell/analysis-codes/rt-cdf/'
    efile = path_ed+mfile[:-10]+'entrains.npy'
    dfile = path_ed+mfile[:-10]+'deposits.npy'
    cfile = path_c+mfile[:-10]+'rtcdf'
    te, me = np.load(efile).T
    td, md = np.load(dfile).T
    m, pm  = np.loadtxt(path_m+mfile).T
    return (te, me, td, md, m, pm, cfile)

def compute_cdf(inputs, bins=bins):
    te, me, td, md, m, pm, cfile = inputs # these come from loader
    cdf = np.zeros(len(bins)-1,dtype=float) # fill this iteratively
    i = 0 
    for mv,p in zip(m, pm): # iterate through all mvalues in question
        i+=1
        Fmv = compute_crt(te, me, td, md, mv)
        cdf+=Fmv*p
    bins = (bins[1:]+bins[:-1])/2.0 # compute the midpoints of the bins 
    data = np.array((bins, cdf)).T
    print(cfile, ' is the save directory') 
    np.save(cfile, data)
    del data,inputs,te,me,md,m,pm,cfile

In [28]:
A_corr=(np.array([66, 67, 70, 133, 133, 133]), 
        np.array([35, 35, 35, 59,  59,  59]), 
        np.array([102,103,104, 105, 106, 107]))
A_corr = np.array(A_corr).T
indices = np.random.choice(np.arange(A_corr.shape[0]),size=2)
A_corr[indices]

array([[133,  59, 105],
       [ 67,  35, 103]])

In [29]:
import numpy as np

A_coor=(np.array([66, 67, 70, 133, 133, 133]), 
        np.array([35, 35, 35, 59,  59,  59]), 
        np.array([102,103,104, 105, 106, 107]))

column = np.random.choice(len(A_coor[0])) # Pick a random column index

coords = np.array(A_coor)[:, column]
print(column, coords)

1 [ 67  35 103]


In [ ]:
nfiles = len(list(os.listdir('/media/kpierce/seagate/reverting-onecell/analysis/m-stats/')))
i=0
print('------------------------------------------------------------------------------')

mfiles = []
for mfile in os.listdir('/media/kpierce/seagate/reverting-onecell/analysis/m-stats/'):
    file1='/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/'+mfile[:-10]+'entrains.npy'
    file2='/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/'+mfile[:-10]+'deposits.npy'
    file3='/media/kpierce/seagate/reverting-onecell/analysis/rtcdf/'+mfile[:-10]+'rtcdf.npy'
    if os.path.isfile(file1) and os.path.isfile(file2) and (not os.path.isfile(file3)):
        i+=1
        print('starting ' + mfile[:-10] + ' at ' + datetime.datetime.utcnow().strftime("%H:%M:%S"))
        print('computation {} of {}.'.format(i,nfiles))
        inputs = loader(mfile)
        compute_cdf(inputs)
        print('finished at ' + datetime.datetime.utcnow().strftime("%H:%M:%S"))
        print('------------------------------------------------------------------------------')

In [ ]:
x = np.load('/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/flow_l-l_1.8-time_500.0hr-entrains.npy')

In [ ]:
files = [f for f in os.listdir('/media/kpierce/seagate/reverting-onecell/analysis/rt-cdf/')]

In [ ]:
files

In [52]:
import numpy as np

A_coor=np.stack((np.array([66, 67, 70, 133, 133, 133]),
            np.array([35, 35, 35, 59,  59,  59]),
            np.array([102,103,104, 105, 106, 107])), axis=0)

k=2
column = np.random.choice(len(A_coor[0]), replace=False, size=k)  

coords = np.array(A_coor)[:, column]
print(coords.T)

[[133  59 107]
 [133  59 105]]


In [50]:
??np.random.choice

In [53]:
import numpy as np

# Parameters
n_t = 10
n_ddl = 3

# Typical dummy M n_ddl-diagonal matrix
x = np.arange(1,31)
x1 = np.arange(1,21)
x2 = np.arange(1,11)
M = np.diag(x) + np.diag(x1, 10) + np.diag(x1, -10) + np.diag(x2, 20) + np.diag(x2, -20)

# First loop remains
for i in range(0,n_t):
    M_i = np.zeros((n_ddl,n_ddl))
    # Optimize the following to get M_i
    for j in range(0,n_ddl,1):
        for k in range(0,n_ddl,1):
            M_i[j,k] = M[j*n_t+i,k*n_t+i]

In [54]:
M_i

array([[ 10.,  10.,  10.],
       [ 10.,  20.,  20.],
       [ 10.,  20.,  30.]])